In [15]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecule
from pennylane import commutator
import matplotlib.pyplot as plt

# 1. 设置分子结构
symbols = ["H", "H"]
coordinates = np.array([[0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.74]])  # 单位 Å

# 2. 分子哈密顿量（cost Hamiltonian）
cost_h, qubits = qml.qchem.molecular_hamiltonian(
    symbols, coordinates, method='pyscf', basis='sto-3g'
)
print("Qubits:", qubits)
print(cost_h)
# 3. Driver Hamiltonian
def build_driver_h(n):
    return qml.Hamiltonian([1.0] * n, [qml.PauliX(i) for i in range(n)])

driver_h = build_driver_h(qubits)
print(driver_h)

# 4. 构造 commutator 哈密顿量
def build_commutator_hamiltonian(H_d, H_c):
    comm_terms = []
    coeffs_d, ops_d = H_d.terms()
    coeffs_c, ops_c = H_c.terms()
    for c1, op1 in zip(coeffs_d, ops_d):
        for c2, op2 in zip(coeffs_c, ops_c):
            try:
                comm = commutator(op1, op2)
                if isinstance(comm, qml.ops.op_math.Sum) and len(comm.operands) == 0:
                    continue
                comm_terms.append(1j * c1 * c2 * comm)
            except Exception as e:
                print(f"跳过 [{op1}, {op2}]：{e}")
                continue
    return sum(comm_terms).simplify()

comm_h = build_commutator_hamiltonian(driver_h, cost_h)

def op_to_pauli_string_and_wires(op):
    valid = {"PauliX": "X", "PauliY": "Y", "PauliZ": "Z", "Identity": "I"}
    ops = extract_ops_recursive(op)
    pauli_word = ""
    wires = []
    for sub_op in ops:
        if sub_op.name not in valid:
            raise ValueError(f"Unsupported sub-op: {sub_op.name}")
        pauli_word += valid[sub_op.name]
        wires.append(sub_op.wires[0])
    return pauli_word, wires

def falqon_layer(beta_k, cost_h, driver_h, delta_t):
    for c, op in zip(cost_h.coeffs, cost_h.ops):
        pauli_word, wires = op_to_pauli_string_and_wires(op)
        decompose_pauli_rot_ion(c * delta_t, pauli_word, wires)

    for c, op in zip(driver_h.coeffs, driver_h.ops):
        pauli_word, wires = op_to_pauli_string_and_wires(op)
        decompose_pauli_rot_ion(c * beta_k * delta_t, pauli_word, wires)


def build_maxclique_ansatz(cost_h, driver_h, delta_t):
    def ansatz(beta, **kwargs):
        layers = len(beta)
        for w in dev.wires:
            qml.Hadamard(wires=w)
        qml.layer(
            falqon_layer,
            layers,
            beta,
            cost_h=cost_h,
            driver_h=driver_h,
            delta_t=delta_t
        )

    return ansatz


def expval_circuit(beta, measurement_h):
    ansatz = build_maxclique_ansatz(cost_h, driver_h, delta_t)
    ansatz(beta)
    return qml.expval(measurement_h)

def falqon( n, beta_1, delta_t, dev):
    comm_h = build_commutator_hamiltonian(driver_h, cost_h)

    cost_fn = qml.QNode(expval_circuit, dev, interface="autograd") # The ansatz + measurement circuit is executable

    beta = [beta_1] # Records each value of beta_k
    energies = [] # Records the value of the cost function at each step

    for i in range(n):
        # Adds a value of beta to the list and evaluates the cost function
        beta.append(-1 * cost_fn(beta, measurement_h=comm_h))  # this call measures the expectation of the commuter hamiltonian
        energy = cost_fn(beta, measurement_h=cost_h)  # this call measures the expectation of the cost hamiltonian
        energies.append(energy)
        print(f"第 {i+1} 步: E = {energy:.8f} Ha")
    return beta, energies

n = 10
beta_1 = 0.0
delta_t = 0.03

dev = qml.device("default.qubit", wires=qubits) # Creates a device for the simulation
res_beta, res_energies = falqon(n, beta_1, delta_t, dev)
plt.plot(range(n+1)[1:], res_energies)
plt.xlabel("Iteration")
plt.ylabel("Cost Function Value")
plt.show()

Qubits: 4
0.7784107703315544 * I([0, 2, 1, 3]) + 0.23718739979583803 * Z(0) + -0.4612495808843312 * Z(2) + 0.1406570062079997 * (Z(0) @ Z(2)) + 0.23718739979583803 * Z(1) + 0.18456105273155665 * (Z(0) @ Z(1)) + 0.04104417944033013 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04104417944033013 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.04104417944033013 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.04104417944033013 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.4612495808843312 * Z(3) + 0.18170118564832982 * (Z(0) @ Z(3)) + 0.1406570062079997 * (Z(1) @ Z(3)) + 0.18170118564832982 * (Z(1) @ Z(2)) + 0.1917875067959805 * (Z(2) @ Z(3))
1.0 * X(0) + 1.0 * X(1) + 1.0 * X(2) + 1.0 * X(3)


NameError: name 'extract_ops_recursive' is not defined